# Exploration14 - 아이유팬이 좋아할 만한 다른 아티스트 찾기
MF 모델을 활용하여 취향에 맞는 영화를 추천해 보자.

---

### 루브릭 평가 기준

1. CSR matrix의 생성 - 사용자와 아이템 개수를 바탕으로 정확한 사이즈로 만들어졌나
2. MF 모델의 훈련으로 인한 적절한 추천 - 사용자와 아이템 벡터 내적 수치가 의미있게 형성되었나
3. 비슷한 영화 찾기와 유저에게 추천하기 과정 진행 - MF모델이 예측한 유저 선호도 및 아이템 간 유사도, 기여도가 의미있게 측정되었나

---

### 목차

0) 필요 모듈 import


1) 데이터 준비 및 전처리
  - 데이터 준비
  - 데이터 탐색
  - 데이터 전처리
  - CSR Matrix 생성
    

2) 모델 학습


3) 결과 확인
  - 비슷한 영화 찾기
  - 내가 좋아할 만한 영화 찾기


4) 회고

## 0. 필요 모듈 import

In [1]:
import pandas as pd
import os
import numpy as np

from scipy.sparse import csr_matrix

from implicit.als import AlternatingLeastSquares

## 1. 데이터 준비 및 전처리

### - 데이터 준비

MovieLens 1M Dataset을 사용하여 프로젝트를 진행하겠다.  
이는 영화 선호도에 대해 별점으로 표현한 데이터인데,  
별점 데이터는 대표적인 explicit 데이터이지만 이번 프로젝트에서는 별점을 시청횟수로 해석하여 implicit 데이터로 간주한다.

In [2]:
rating_file_path = './data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'ratings', 'timestamp']

ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
original_data_size = len(ratings)

ratings.head()

,user_id,movie_id,ratings,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


유저가 별점을 3점 미만으로 준 영화는 선호하지 않다고 가정하고 이는 모두 제외해준다.

In [3]:
# 3점 이상만 남긴다.
ratings = ratings[ratings['ratings']>=3]
filtered_data_size = len(ratings)

print(f'original_data_size: {original_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / original_data_size:.2%}')

original_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


별점을 시청횟수로 해석하기로 하였으므로 컬럼의 이름을 바꾼다.

In [4]:
# ratings 컬럼의 이름을 counts로 바꾼다.
ratings.rename(columns={'ratings':'counts'}, inplace=True)
ratings['counts']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: counts, Length: 836478, dtype: int64

ratings는 이미 인덱싱까지 완료된 사용자-영화-평점 데이터였으므로 영화 제목을 보기 위해 메타 데이터를 불러온다.

In [5]:
# 영화 제목을 보기 위해 메타 데이터를 불러온다.
movie_file_path =  './data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


### - 데이터 탐색

ratings와 movies 데이터에 movie_id가 공통적으로 존재, 이를 기준으로 두 데이터를 합쳐 분석해보겠다.

In [6]:
data = pd.merge(ratings, movies)
data.head()

,user_id,movie_id,counts,timestamp,title,genre
0,1,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama
1,2,1193,5,978298413,One Flew Over the Cuckoo's Nest (1975),Drama
2,12,1193,4,978220179,One Flew Over the Cuckoo's Nest (1975),Drama
3,15,1193,4,978199279,One Flew Over the Cuckoo's Nest (1975),Drama
4,17,1193,5,978158471,One Flew Over the Cuckoo's Nest (1975),Drama


분석에 필요한 데이터만 남기고 나머지는 삭제한다.

In [7]:
# 사용하는 컬럼만 남겨준다.
using_cols = ['user_id', 'counts', 'title']
data = data[using_cols]
data.head(10)

,user_id,counts,title
0,1,5,One Flew Over the Cuckoo's Nest (1975)
1,2,5,One Flew Over the Cuckoo's Nest (1975)
2,12,4,One Flew Over the Cuckoo's Nest (1975)
3,15,4,One Flew Over the Cuckoo's Nest (1975)
4,17,5,One Flew Over the Cuckoo's Nest (1975)
5,18,4,One Flew Over the Cuckoo's Nest (1975)
6,19,5,One Flew Over the Cuckoo's Nest (1975)
7,24,5,One Flew Over the Cuckoo's Nest (1975)
8,28,3,One Flew Over the Cuckoo's Nest (1975)
9,33,5,One Flew Over the Cuckoo's Nest (1975)


pandas.DataFrame.nunique( )는 특정 컬럼에 포함된 유니크한 데이터의 개수를 알아보는데 유용하다.  
이를 이용하여 유저 수와 영화 수를 확인해보자.

In [8]:
# 유저 수
data['user_id'].nunique()

6039

In [9]:
# 영화 수
data['title'].nunique()

3628

유저들에게 인기가 많은 영화 30개를 뽑아보자.

In [10]:
# 인기 많은 영화
movie_count = data.groupby('title')['user_id'].count()
movie_count.sort_values(ascending=False).head(30)

title
American Beauty (1999)                                   3211
Star Wars: Episode IV - A New Hope (1977)                2910
Star Wars: Episode V - The Empire Strikes Back (1980)    2885
Star Wars: Episode VI - Return of the Jedi (1983)        2716
Saving Private Ryan (1998)                               2561
Terminator 2: Judgment Day (1991)                        2509
Silence of the Lambs, The (1991)                         2498
Raiders of the Lost Ark (1981)                           2473
Back to the Future (1985)                                2460
Matrix, The (1999)                                       2434
Jurassic Park (1993)                                     2413
Sixth Sense, The (1999)                                  2385
Fargo (1996)                                             2371
Braveheart (1995)                                        2314
Men in Black (1997)                                      2297
Schindler's List (1993)                                  2257
Pr

유저 별 몇 편의 영화를 봤는지에 대한 통계를 확인한다.

In [11]:
user_count = data.groupby('user_id')['title'].count()
user_count.describe()

count    6039.000000
mean      138.512668
std       156.241599
min         1.000000
25%        38.000000
50%        81.000000
75%       177.000000
max      1968.000000
Name: title, dtype: float64

#### 내가 좋아하는 영화 정보 추가  

모델 검증을 위해 사용자 초기 정보를 추가해 준다.  
user_id : chaen

In [12]:
my_favorite = ['Terminator 2: Judgment Day (1991)', 'Back to the Future (1985)', 'Ghostbusters (1984)',
               'E.T. the Extra-Terrestrial (1982)', 'Sixth Sense, The (1999)']

# 'chaen'이라는 user_id가 위 영화를 5회씩 보았다고 가정
my_playlist = pd.DataFrame({'user_id': ['chaen']*5, 'title': my_favorite, 'counts':[5]*5})

if not data.isin({'user_id':['chaen']})['user_id'].any():  # user_id에 'chaen'이라는 데이터가 없다면
    data = data.append(my_playlist)                           # 위에 임의로 만든 my_favorite 데이터를 추가
    
data.tail(10)

,user_id,counts,title
836473,5851,5,One Little Indian (1973)
836474,5854,4,Slaughterhouse (1987)
836475,5854,3,"Promise, The (Versprechen, Das) (1994)"
836476,5938,4,"Five Wives, Three Secretaries and Me (1998)"
836477,5948,5,Identification of a Woman (Identificazione di ...
0,chaen,5,Terminator 2: Judgment Day (1991)
1,chaen,5,Back to the Future (1985)
2,chaen,5,Ghostbusters (1984)
3,chaen,5,E.T. the Extra-Terrestrial (1982)
4,chaen,5,"Sixth Sense, The (1999)"


### - 데이터 전처리

pandas.DataFrame.unique( )은 특정 컬럼에 포함된 유니크한 데이터만 모아 준다.   
이를 이용해 user_id와 title에 대하여 indexing 작업을 수행한다.

In [13]:
# 고유한 유저, 영화를 찾아내는 코드
user_unique = data['user_id'].unique()
movie_unique = data['title'].unique()

# 유저, 영화 indexing 하는 코드
user_to_idx = {v:k for k,v in enumerate(user_unique)}
movie_to_idx = {v:k for k,v in enumerate(movie_unique)}

In [14]:
print(user_to_idx['chaen'])    # 마지막으로 추가된 유저이니 6040번째 유저이므로 6039이 나와야 한다. 
print(movie_to_idx['Ghostbusters (1984)'])

6039
243


In [15]:
# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 
temp_user_data = data['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(data):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    data['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

# movie_to_idx을 통해 movie 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_movie_data = data['title'].map(movie_to_idx.get).dropna()
if len(temp_movie_data) == len(data):
    print('title column indexing OK!!')
    data['title'] = temp_movie_data
else:
    print('title column indexing Fail!!')

data

user_id column indexing OK!!
title column indexing OK!!


,user_id,counts,title
0,0,5,0
1,1,5,0
2,2,4,0
3,3,4,0
4,4,5,0
...,...,...,...
0,6039,5,92
1,6039,5,22
2,6039,5,243
3,6039,5,26


### - CSR Matrix 생성 

메모리 사용량을 최소화하기 위해 행렬을 압축하여 표현할 수 있는 CSR Matrix를 생성한다. 

In [16]:
num_user = data['user_id'].nunique()
num_movie = data['title'].nunique()

csr_data = csr_matrix((data.counts, (data.user_id, data.title)), shape = (num_user, num_movie))
csr_data

<6040x3628 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Row format>

## 2. 모델 학습

암묵적 데이터를 사용하는 다양한 모델을 빠르게 학습할 수 있는 implicit 패키지에 구현된 als 모델을 사용하여 학습을 진행한다.

In [17]:
# implicit 라이브러리에서 권장하고 있는 부분
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

AlternatingLeastSquares 클래스의 \__init__ 파라미터

1. factors : 유저와 아이템의 벡터를 몇 차원으로 할 것인지
2. regularization : 과적합을 방지하기 위해 정규화 값을 얼마나 사용할 것인지
3. use_gpu : GPU를 사용할 것인지
4. iterations : epochs와 같은 의미입니다. 데이터를 몇 번 반복해서 학습할 것인지

In [18]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=100,
                                    regularization=0.01,
                                    use_gpu=False,
                                    iterations=30,
                                    dtype=np.float32)

In [19]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose하여 사용)
csr_data_transpose = csr_data.T
csr_data_transpose

<3628x6040 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [20]:
# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/30 [00:00<?, ?it/s]

나의 선호도를 추가한 user_id:chaen 데이터에 대하여 벡터를 어떻게 만들고 있는지, 특정 영화에 대해서는 벡터를 어떻게 만들고 있는지 확인해보자.  

In [21]:
chaen, ghostbusters = user_to_idx['chaen'], movie_to_idx['Ghostbusters (1984)']
chaen_vector, ghostbusters_vector = als_model.user_factors[chaen], als_model.item_factors[ghostbusters]

In [22]:
chaen_vector

array([-0.10715943, -0.07665382,  0.6862064 ,  0.45160556, -0.10842567,
        0.09913171,  0.3599588 , -0.04734577,  0.6727693 ,  0.05101774,
       -0.15247065, -0.63073665, -0.01791093,  0.06157181, -0.02197258,
       -0.6150168 , -0.06350518,  0.29245275, -0.32628062,  0.33736575,
       -0.6137358 , -0.23726933,  0.5903623 , -0.5193583 ,  0.16199063,
       -0.29614183, -0.41350955, -0.11821482, -0.47660074, -0.8108594 ,
       -1.1413038 ,  0.68568885,  0.3982033 ,  0.01952579, -0.8365466 ,
        0.17747998,  0.81729436,  0.4762849 ,  0.95177865,  0.60231894,
       -0.22309636, -0.70968443,  0.2614132 , -0.21286575, -1.1211594 ,
        0.30254018,  0.32906932,  0.03215273, -0.03333445,  0.12247662,
        0.59684545, -0.22724687, -0.3342536 ,  0.4428008 ,  0.02151689,
       -0.11933529, -0.8833497 , -0.42201662,  0.03873072,  0.4684013 ,
       -0.28241637, -0.11065447, -0.57594913, -0.49711084, -0.13113892,
        0.53644764,  0.5171762 ,  0.11302017,  0.20850894,  0.46

In [23]:
ghostbusters_vector

array([ 2.05266401e-02,  3.39976535e-03,  3.42092142e-02,  1.25162592e-02,
        4.74924018e-05,  1.98606178e-02,  4.51107323e-02,  1.26385614e-02,
       -6.34205341e-03,  3.12709361e-02, -5.98976342e-03, -3.15249935e-02,
       -5.33249509e-03,  1.14297131e-02,  1.15609691e-02, -1.35865621e-02,
        3.70968468e-02, -2.12686080e-02, -1.93170272e-02,  3.84958088e-02,
       -1.78913921e-02, -2.36714352e-03,  8.55903374e-04, -4.14416380e-02,
       -1.15506956e-02, -1.29607497e-02,  6.61822781e-03,  3.30612659e-02,
       -1.88940614e-02, -1.29072815e-02, -4.85753156e-02, -9.82843526e-03,
       -3.57963610e-03, -1.25875836e-02,  3.32339900e-03, -2.73671467e-04,
        3.06391027e-02,  2.85288561e-02,  2.34022886e-02,  3.06983311e-02,
       -5.88418916e-03, -1.70625746e-02,  8.25099945e-02,  1.14474371e-02,
       -1.41150625e-02,  1.14790779e-02,  1.83136258e-02,  9.39228944e-03,
        1.46051100e-03,  7.19498917e-02,  3.93852219e-02, -1.13856690e-02,
       -6.73352089e-03,  

이 두 벡터를 내적하면 나의 선호도를 알 수 있다.

In [24]:
np.dot(chaen_vector, ghostbusters_vector)

0.58505446

별점을 5개 준 것 치곤 높은 선호도라고 할 수는 없는 것 같지만 그래도 이정도면 선호하는 영화라고 할 수 있을 것 같다.

## 3. 결과 확인

### - 비슷한 영화 찾기

AlternatingLeastSquares 클래스에 구현되어 있는 similar_items 메서드를 이용하여 학습된 모델이 입력된 영화에 대해 비슷한 영화를 찾을 수 있도록 한다.

In [25]:
#movie_to_idx 를 뒤집어, index로부터 영화 제목을 얻는 dict를 생성
idx_to_movie = {v:k for k,v in movie_to_idx.items()}

In [26]:
# 비슷한 영화를 찾아주는 함수
def get_similar_movie(movie_title: str):
    movie_id = movie_to_idx[movie_title]
    similar_movie = als_model.similar_items(movie_id)
    similar_movie = [idx_to_movie[i[0]] for i in similar_movie]
    return similar_movie

In [27]:
get_similar_movie('Jurassic Park (1993)')

['Jurassic Park (1993)',
 'Men in Black (1997)',
 'Terminator 2: Judgment Day (1991)',
 'Total Recall (1990)',
 'Matrix, The (1999)',
 'Braveheart (1995)',
 'Lost World: Jurassic Park, The (1997)',
 'Schlafes Bruder (Brother of Sleep) (1995)',
 'Independence Day (ID4) (1996)',
 'Mission: Impossible (1996)']

### - 내가 좋아하는 영화 찾기

AlternatingLeastSquares 클래스에 구현되어 있는 recommend 메서드를 이용하여 내가 좋아할 만한 영화를 추천받는다.

In [28]:
user = user_to_idx['chaen']

movie_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True) 
                                                 # filter_already_liked_items 는 유저가 이미 평가한 아이템은 제외하는 Argument
movie_recommended

[(124, 0.4270349),
 (107, 0.41389352),
 (99, 0.31886056),
 (200, 0.31102312),
 (44, 0.3073797),
 (64, 0.29468012),
 (141, 0.29052576),
 (19, 0.28695035),
 (121, 0.2835486),
 (670, 0.27736205),
 (175, 0.26761374),
 (60, 0.26720667),
 (117, 0.25914904),
 (582, 0.24680988),
 (91, 0.24464147),
 (15, 0.2430883),
 (13, 0.23786795),
 (87, 0.23102692),
 (1144, 0.21713956),
 (126, 0.2138846)]

In [29]:
for i in movie_recommended:
    print("{} : {:04.3f}".format(idx_to_movie[i[0]], i[1]))

Matrix, The (1999) : 0.427
Jurassic Park (1993) : 0.414
American Beauty (1999) : 0.319
Terminator, The (1984) : 0.311
Star Wars: Episode IV - A New Hope (1977) : 0.307
Star Wars: Episode VI - Return of the Jedi (1983) : 0.295
Fugitive, The (1993) : 0.291
Big (1988) : 0.287
Silence of the Lambs, The (1991) : 0.284
Galaxy Quest (1999) : 0.277
Men in Black (1997) : 0.268
Star Wars: Episode I - The Phantom Menace (1999) : 0.267
Star Wars: Episode V - The Empire Strikes Back (1980) : 0.259
Beetlejuice (1988) : 0.247
Close Encounters of the Third Kind (1977) : 0.245
Airplane! (1980) : 0.243
Ferris Bueller's Day Off (1986) : 0.238
Braveheart (1995) : 0.231
Bull Durham (1988) : 0.217
Shakespeare in Love (1998) : 0.214


추천하는 영화라기에는 예상 선호도가 낮긴하지만 15개 영화를 추천 받았다.  
내가 평가한 어떤 영화가 이러한 추천을 하는데 기여했는지 알아보자.  
AlternatingLeastSquares 클래스에 구현된 explain 메서드를 사용하면 내가 기록을 남긴 데이터 중 이 추천에 기여한 정도를 확인할 수 있다.

In [30]:
matrix = movie_to_idx['Matrix, The (1999)']
explain = als_model.explain(user, csr_data, itemid = matrix)

In [31]:
[(idx_to_movie[i[0]], i[1]) for i in explain[1]]

[('Terminator 2: Judgment Day (1991)', 0.2777048609586838),
 ('Sixth Sense, The (1999)', 0.0944047438973886),
 ('Back to the Future (1985)', 0.04417907733217705),
 ('E.T. the Extra-Terrestrial (1982)', 0.011431467938676915),
 ('Ghostbusters (1984)', -0.004576746292281404)]

## 4. 회고

#### - 영화를 잘 몰라요..

데이터셋에 포함된 영화들이 1990대 개봉된 영화가 많은 것 같은데 내가 잘 모르는 영화가 대부분이라 인기 많은 영화 중 제목을 들어본 영화를 나의 선호로 넣어주었다.  
선호 영화가 좀 일관적이지 않았는지, 모델이 확실하게 높은 수치로 추천을 해주지 못한 것 같다.  
또한 내가 잘 모르는 영화들이다 보니 추천해준 영화가 진짜 나의 취향에 맞는 것인지 판단을 못해 아쉽다.  
다음에는 내가 흥미있는 주제와 관련된 데이터를 사용해보면 좋을 것 같다.  


#### -  사람의 취향을 수학적으로 표현하기

NLP 노드도 그렇고 이번 노드 같은 추천 시스템 등은 사람의 감정이나 선호 같은 무형의 것들을 벡터를 이용해 수학적으로 표현해야 한다.  
앞으로 진짜 사람같은 인공지능을 만들기 위해서는 이 부분에 대한 이해가 필수적일 것 같은데 나에게는 아직은 조금 어렵다.  
학문적으로 선형대수학을 공부할 때는 큰 어려움이 없었는데 현실에 존재하는 것들을 벡터로 옮기는 것은 또 다른 이야기 인 것 같다.  